### Beginning of the Assignment - exploration

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
#pd.set_option('max_columns', 200)

### PHIL DATA SET

In [ ]:
character_nicknames_df = pd.read_csv('datasetscharacter_nicknames.csv')

In [ ]:
details_df = pd.read_csv('datasets/details.csv')

In [ ]:
favs_df = pd.read_csv('datasets/favs.csv')


In [ ]:
person_details_df = pd.read_csv('datasets/person_details.csv')


In [ ]:
person_alternate_names_df = pd.read_csv('datasets/person_alternate_names.csv')


In [ ]:
person_anime_works_df = pd.read_csv('datasets/person_anime_works.csv')


In [ ]:
stats_df = pd.read_csv('datasets/stats.csv')


### DENIS DATA SET

In [5]:
ratings_df = pd.read_csv('datasets/ratings.csv')

In [ ]:
characters_df = pd.read_csv('datasets/characters.csv')

In [ ]:
character_anime_works_df = pd.read_csv('datasets/character_anime_works.csv')

In [ ]:
person_voice_works_df = pd.read_csv('datasets/person_voice_works.csv')

In [ ]:
profiles_df = pd.read_csv('datasets/profiles.csv')

In [ ]:
recommendations_df = pd.read_csv('datasets/recommendations.csv')

# GUIDELINES
### BEFORE STARTING
Use Conda in order to do the correct setup

When we deliver the project, we need to tell the lecturer to run the jupyter notebook before running the TWEB part. How? Write it in the Report
Give instructions in order to make it run properly

#TODO to write on the report:
Which are the cool things, problems, map names not allined so we had to normalize them and so on

Things to do for each dataset:
1. Give it a look with .head and/or .tail
2. .describe and check if all the numeric values make sense (e.g. year=300 makes no sense in our context)
3. Check the format: objects to date if we need it. Check if all the dates are in the same format: us or eu
    also check if there's any 29/02/2013. It doesn't exist right? Maybe this is too much lol
4. Check for duplicates
5. CHECK FOR CORRELATION: df.corr() (e.g. with longer duration, there are more actors)
#TODO
6. ADD or Remove columns?
7. GROUPBY selects the elements and makes group out of it, combines the numeric fields of each specific group
#TODO We could use it grouping for language and looking at how many anime are made in japan, stating it's the first country where the culture of doing (and watching) anime is SO big
8. Aggregations: we can apply multiple different aggregated functions (e.g. for the first column you sum the data, for the second you do the average and so on)
9. Transformations: apply operations and return results aligned with the original DF

10. Removing NaNs is wrong in general because Pandas will skip it.
We do it when? Is it safe to remove NaNs rows if EVERY field in the row is empty? I hope so lol
BE CAREFUL if they are foreign keys: for example, if a person has a nan in the "anime he worked in" field, it shouldn't be dropped
NEVER replace with invalid values (e.g. -1)
IF we use df.dropna(subset=["name"],inplace=True)
the inplace means that the df itself is modified and will result in the one without the na. Without "inplace=true" you'll need to assign the result to another df (or the same) 


Proviamo i plot? df.plot()

11. Check if data are consistent (e.g. normalizing names of countries and/or numeric fields, describing them and checking what they are)

12. Normalize data types all in the same place (e.g. all the dates in the same cell)


BONUS: NEVER USE LOOP FOR, NEVER DUPLICATE DATA (unless necessary)


##### First look

In [ ]:
character_nicknames_df

In [ ]:
character_nicknames_df.head()

In [ ]:
character_nicknames_df.columns
#will list all the columns. Not necessary here but kept as a pattern to follow with the following files

In [ ]:
character_nicknames_df.dtypes

### Data preparation (cleaning)


##### On the first dataset we may need to check for duplicates.
What does that mean? We have 102 rows that are duplicated over a 37080 rows dataset.
Why is that? Are there some characters that have multiple nicknames, so they are repeated in the dataset?

In [ ]:
character_nicknames_df.info()

In [ ]:
character_nicknames_df.describe(include='all')   

In [ ]:
character_nicknames_df.loc[character_nicknames_df.duplicated()]
#by default will give us the second

Mhh they're somehow different so yeah, the same character could have different nicknames.
We want to drop though the ones that are exactly the same.

In [ ]:
#this way we drop the duplicates on the first dataset

# OLD VERSION 
# character_nicknames_df = character_nicknames_df.loc[~character_nicknames_df.duplicated()].reset_index(drop=True).copy()

#we don't need to use a subset here because there are just 2 columns

#why don't we just use drop_duplicates()?
character_nicknames_df.drop_duplicates(keep='first', inplace=True)

Let's check for nan values

In [ ]:
character_nicknames_df[character_nicknames_df.isna().any(axis=1)]


In [ ]:
#cleaning the df from nan values
character_nicknames_df.dropna(inplace=True)


In [ ]:
# OLD VERSION
# character_nicknames_df.to_csv("datasets/character_nicknames_cleaned.csv", index=False, line_terminator="\n")
# With this command, the cleaned dataset is bigger than the original
# This is because on Windows line endings occupy 2 bytes instead of 1
# In order to fix it and have an actual smaller file that matches with macOS / Linux, we add the argument 
# lineterminator, which in older versions of pandas may not work, and must be replaced with line_terminator
character_nicknames_df.to_csv("datasets/character_nicknames_cleaned.csv", index=False, lineterminator="\n")

In [ ]:
#TODO put this somewhere else
small_datasets = {
    "character_nicknames": character_nicknames_df,
    "details": details_df,
    "person_alternate_names": person_alternate_names_df,
    "person_details": person_details_df,
    "stats": stats_df,
    "ratings": ratings_df,
    "characters": characters_df,
    "person_voice_works": person_voice_works_df,
    "profiles": profiles_df,
}

for name, df in small_datasets.items():
    df.to_csv(f"{name}_cleaned.csv", index=False, lineterminator="\n")


## SECOND DATASET

##### On the second dataset we may need to check for missing values and/or inconsistent values, since there are no duplicates

In [ ]:
details_df
#anime details
#japanes title could be dropped?
#members stand for how many users have this anime added to their list.
#explicit_genres is empty so can be removed
#licensor and streaming are mostly empty. Do we care?

In [ ]:
# we want to see what are "type"
details_df['type'].value_counts()

In [ ]:
details_df.loc[details_df.duplicated(subset=['url'], keep='first')]

In [ ]:
details_df.query('year>2025')

In [ ]:
details_df.describe()

In [ ]:
details_df[['start_date','season']].query("season.notna()")
#season can be removed? Do we care about the season? We can "calculate" it from the "start_date" field

In [ ]:
details_df.query("episodes > 2500")

In [ ]:
details_df[["start_date", "end_date"]] = details_df[["start_date", "end_date"]].apply(
    pd.to_datetime, errors="coerce"
)

In [ ]:
details_df[["scored_by", "rank", "episodes", "year"]] = (
    details_df[["scored_by", "rank", "episodes", "year"]].astype("Int64")
)


In [ ]:
details_df.dtypes
#scored_by, rank, episodes, year can be an int instead of a float
#start and end dates are not objects but dates
#do we need to swap the empty [] with Nan or not? WE should in order to be able to use the .isna() method and other pandas methods


### THIRD DATASET

In [ ]:
favs_df

In [ ]:
# we want to see what are "fav_type"
favs_df['fav_type'].value_counts()

In [ ]:
favs_df.isna().sum()


In [ ]:
favs_df.dtypes

In [ ]:
favs_df.duplicated().sum()
#there are no duplicates


### FOURTH DATASET

In [ ]:
person_alternate_names_df

In [ ]:
person_alternate_names_df.dtypes

In [ ]:
person_alternate_names_df.isna().sum()
person_alternate_names_df[person_alternate_names_df.isna().any(axis=1)]


In [ ]:
person_alternate_names_df.dropna(inplace=True)



In [ ]:
# person_alternate_names_df.loc[person_alternate_names_df['person_mal_id'].duplicated()]
person_alternate_names_df[person_alternate_names_df.duplicated(subset=['person_mal_id','alt_name'], keep=False)].sort_values(['person_mal_id','alt_name'])


In [ ]:
person_alternate_names_df.drop_duplicates(keep='first', inplace=True)

### FIFTH DATASET

In [ ]:
person_details_df

In [ ]:
person_details_df.loc[person_details_df['person_mal_id'].duplicated()]
#we found that the duplicates differ for the "relevant_location" field, which has no interest for us so we drop the duplicates

In [ ]:
person_details_df.drop_duplicates(subset=['person_mal_id', 'url', 'name'], keep='first', inplace=True)


In [ ]:
person_details_df.dtypes
#we need to change birthday from object to data

In [ ]:
person_details_df["birthday"] = pd.to_datetime(person_details_df["birthday"], errors='coerce')

In [ ]:
person_details_df["birthday"].min(), person_details_df["birthday"].max()
#makes sense because they're just composers of used music in anime

In [ ]:
person_details_df.isna().sum()
#we have to check the nan values

In [ ]:
person_details_df[person_details_df["name"].isna()]


We can join the two tables person_details_df and person_alternate_names_df having the keys that match.
Putting the alternate names in a new column called alt_name and having a list of those inside

### SIXTH DATASET

In [ ]:
person_anime_works_df

In [ ]:
person_anime_works_df.dtypes
#the types are correct

In [ ]:
person_anime_works_df.isna().sum()
#There's no nan value

### SEVENTH DATASET

In [ ]:
stats_df.filter(regex="_votes$").astype("Int64")
stats_df[stats_df.filter(regex="_votes$").columns] = (
    stats_df.filter(regex="_votes$").astype("Int64")
)

In [ ]:
stats_df.isna().sum()
#there are 430 series without any votes

### EIGHTH DATASET 

In [ ]:
ratings_df

In [ ]:
ratings_df.columns

In [ ]:
# we have to understand the sense of "num_watched_episodes" and the link with "is_rewatching"
ratings_df.query('is_rewatching == 1')

In [ ]:
ratings_df.dtypes

In [ ]:
# change "is_rewatching" from float to Int8, to save memory
ratings_df["is_rewatching"] = ratings_df["is_rewatching"].astype("Int8") 

In [ ]:
ratings_df[["anime_id","score","num_watched_episodes"]].agg(["min", "max"])


In [ ]:
# to save up some memory, we can change "anime_id" from Int64 to Int32 because there are no anime with id > 2,147,483,647
ratings_df["anime_id"] = ratings_df["anime_id"].astype("Int32")

In [ ]:
#same with "score", which goes from 1 to 10
ratings_df["score"] = ratings_df["score"].astype("Int8")

In [ ]:
#same with "num_watched_episodes", which can't go over 2 billions
ratings_df["num_watched_episodes"] = ratings_df["num_watched_episodes"].astype("Int32")

In [ ]:
ratings_df[ratings_df.duplicated(subset=['username','anime_id'], keep=False)].sort_values(['username','anime_id'])

In [ ]:
# usually we should drop all the occurrence of a duplicate and keep the first
# in this case though, it looks like the latest occurence is the most updated one, contaning more info than the first one, so we drop the first one
ratings_df.drop_duplicates(subset=['username', 'anime_id'], keep='last', inplace=True)

We had just 6 duplicates having the same username and anime_id

In [ ]:
# we check for Nan values.
#TODO
# if it is necessary check if the num_watched_episodes is greater than number of episodes of anime, we can remove the Nan values and put one or zero. 
ratings_df.isna().sum()

In [ ]:
# drop "username" with Nan values?
#TODO
ratings_df[ratings_df['username'].isna()]

Check this username that there is in the profiles_df

In [ ]:
first_chunk = True
for chunk in pd.read_csv("datasets/ratings.csv", chunksize=2_000_000):
    chunk["is_rewatching"] = chunk["is_rewatching"].astype("Int8")
    chunk["anime_id"] = chunk["anime_id"].astype("Int32")
    chunk["score"] = chunk["score"].astype("Int8")
    chunk["num_watched_episodes"] = chunk["num_watched_episodes"].astype("Int32")

    chunk.to_csv(
        "datasets/ratings_half_cleaned.csv", mode="w" if first_chunk else "a", 
        index=False, header=first_chunk,lineterminator="\n"
    )
    first_chunk = False

ratings_cleaned_df = pd.read_csv("datasets/ratings_half_cleaned.csv")
ratings_cleaned_df.drop_duplicates(subset=["username", "anime_id"], keep="last", inplace=True)
ratings_cleaned_df.to_csv("datasets/ratings_cleaned.csv", index=False, lineterminator="\n")

# This takes sooo long I'm not sure it's worth it. More than 12 minutes on my most powerful machine.
# The cleaned version is about the same size as the original one and we save just a bit of memory when loading it (10% less)
# This is because the file will be read and written twice, just for a small gain in memory usage
# Why twice? Because if we drop duplicates whithin each chunk while reading it, we may miss duplicates that are in different chunks.

In [6]:
dtypes = {
    "is_rewatching": "Int8",
    "anime_id": "Int32",
    "score": "Int8",
    "num_watched_episodes": "Int32",
}

ratings = pd.read_csv("datasets/ratings.csv", dtype=dtypes, low_memory=False)

ratings["username"] = ratings["username"].astype("category")

ratings.drop_duplicates(subset=["username", "anime_id"], keep="last", inplace=True)

ratings.to_csv("datasets/ratings_cleaned.csv", index=False, lineterminator="\n")

# converted high-cardinality string columns with many repeated values 
# (e.g., username) to the category type to reduce memory usage.
# This representation stores each distinct value once and references it via integer 
# codes, allowing the dataset to be processed efficiently on machines with limited RAM.
# THIS "ONLY" TAKES 5 MINUTES TO RUN
# It actually causes more ram usage, not sure why. Is it because of the conversion to category? 
# Or because we try to do all these operations in the same cell?

### NINTH DATASET

In [ ]:
characters_df

In [ ]:
# check types of dataset columns
characters_df.dtypes

In [ ]:
# change "character_mal_id" and "favorites" from float to int
characters_df["character_mal_id"] = characters_df["character_mal_id"].astype("Int64")
characters_df["favorites"] = characters_df["favorites"].astype("Int64")

In [ ]:
characters_df.describe()

In [ ]:
# we have only 2 rows where all columns are Nan, the rows with Nan values in "name_kanji" and "about" we shouldn't drop because they have other values that are important.
characters_df.isna().sum()

In [ ]:
# here we want to check if the Nan values are concentrate in only two rows
characters_df[characters_df['character_mal_id'].isna()]

In [ ]:
characters_df[characters_df['name_kanji'].isna()]
# TODO could we analyze how a missing kanji states something about the character? 1/4 of the characters don't have it
# I guess that kanji name is missing for non-Japanese characters and/or for minor characters.	

In [ ]:
# Apart "name_kanji" and "about" the others Nan values are concentrate in two rows so we drop the two rows with all columns Nan
characters_df.dropna(how='all', inplace=True)


In [ ]:
# we want to see all duplicates to understand if we have to drop or not
characters_df.loc[characters_df.duplicated(subset=['character_mal_id', 'url', 'name'], keep=False)]

In [ ]:
# we drop the duplicates because they have all same values 
characters_df.drop_duplicates(subset=['character_mal_id', 'url', 'name'], keep='first', inplace=True)

### TENTH DATASET

In [ ]:
# role of character anime
character_anime_works_df

In [ ]:
# check types of columns
character_anime_works_df.dtypes

In [ ]:
# check the number of Nan value
character_anime_works_df.isna().sum()

In [ ]:
# check the number of duplicates
character_anime_works_df.loc[character_anime_works_df.duplicated(subset=['anime_mal_id', 'character_mal_id'])]

There is no need to clean this dataset 

### ELEVENTH DATASET

In [ ]:
person_voice_works_df

In [ ]:
person_voice_works_df['language'].value_counts()

In [ ]:
person_voice_works_df['language'].sum()

In [ ]:
#I want to group by language and see the different languages available in the dataset.
person_voice_works_df.groupby('language').size()

In [ ]:
person_voice_works_df.dtypes

In [ ]:
person_voice_works_df.isna().sum()

In [ ]:
# check if the duplicates are in all columns
person_voice_works_df.loc[person_voice_works_df.duplicated(keep=False)]

In [ ]:
# drop the duplicates because they have all same values
person_voice_works_df.drop_duplicates(keep='first', inplace=True)

### TWELFTH DATASET

In [ ]:
# Should we delete the last five columns?
# TODO
profiles_df

In [ ]:
# check if the types are right for each field
profiles_df.dtypes

In [ ]:
# trying things with the date
# BEFORE running the .to_datetime command, ishiyama_yumi has Jul 24 bday
profiles_df.loc[44]

In [ ]:
# change types of columns "birthday" and "joined" from object to date and the others columns that they should be int
profiles_df["birthday"] = pd.to_datetime(profiles_df["birthday"], errors='coerce')
profiles_df["joined"] = pd.to_datetime(profiles_df["joined"], errors='coerce')

In [ ]:
# AFTER running the .to_datetime command, ishiyama_yumi has NaT birthday
profiles_df.loc[44]

In [ ]:
profiles_df["birthday"].min(), profiles_df["birthday"].max()


In [ ]:
weird_birthdays = profiles_df[
    (profiles_df["birthday"] < "1900-01-01") |
    (profiles_df["birthday"] > "2025-12-31")
]

weird_birthdays


In [ ]:
#todo
# Just noticed some birthdays have no year but just day and month, how could we manage those?
# I guess with errors='coerce', when we change type from object to date, the year is set to a default one (e.g. 1800 or 1900)

In [ ]:
profiles_df.loc[
    profiles_df["birthday"] > profiles_df["joined"],
    ["birthday", "joined"]
]


In [ ]:
profiles_df["joined"].min(), profiles_df["joined"].max()

In [ ]:
profiles_df["birthday"].dt.year.value_counts().sort_index().head(30)

In [ ]:
profiles_df[profiles_df["birthday"].dt.year == 1930]

In [ ]:
# decided to remove the birthdays of people older than 100 when they joined the website
mask_too_old = (profiles_df["joined"] - profiles_df["birthday"]).dt.days / 365.25 > 100
profiles_df.loc[mask_too_old, "birthday"] = pd.NaT


In [ ]:
mask_too_young = (profiles_df["joined"] - profiles_df["birthday"]).dt.days / 365.25 < 3
mask_too_young


In [ ]:
profiles_df.isna().sum()
#TODO one guy has null username
# We'll put a progressive unkown_user[] label on it, so that for future data with missing username we could go on with it

In [ ]:
# check if there is any duplicate on "username"
profiles_df.loc[profiles_df.duplicated(subset=['username'], keep='first')]
# none found

### THIRTEENTH DATASET

In [ ]:
recommendations_df

In [ ]:
recommendations_df.dtypes

In [ ]:
recommendations_df.isna().sum()

In [ ]:
recommendations_df.loc[profiles_df.duplicated(keep='first')]

# TO KEEP

In [ ]:
# 1st
# dropping duplicates
character_nicknames_df.drop_duplicates(keep='first', inplace=True)
# dropping nan values
character_nicknames_df.dropna(inplace=True)

# 2nd
# no need to clean from duplicates nor missing values
# change column types from object to datetime
details_df[["start_date", "end_date"]] = details_df[["start_date", "end_date"]].apply(
    pd.to_datetime, errors="coerce"
)
# change column types from object to Int64
details_df[["scored_by", "rank", "episodes", "year"]] = (
    details_df[["scored_by", "rank", "episodes", "year"]].astype("Int64")
)

# 3rd
# no need to clean


# 4th
#dropping nan values because they don't give any useful information
person_alternate_names_df.dropna(inplace=True)
# dropping duplicates
person_alternate_names_df.drop_duplicates(keep='first', inplace=True)


# 5th
# Dropping duplicates choosing to keep the first occurrence. They only differ for the "relevant_location" field which has no interest for us
person_details_df.drop_duplicates(subset=['person_mal_id', 'url', 'name'], keep='first', inplace=True)
# change column types from object to datetime
person_details_df["birthday"] = pd.to_datetime(person_details_df["birthday"], errors='coerce')

# There are two weird rows with nan values in most columns but we don't drop them because they may be linked to other datasets
# Could we join the two tables person_details_df and person_alternate_names_df having the keys that match.
# Putting the alternate names in a new column called alt_name and having a list of those inside
# TODO


# 6th
# no need to clean


# 7th
# change column types to save memory
stats_df.filter(regex="_votes$").astype("Int64")
stats_df[stats_df.filter(regex="_votes$").columns] = (
    stats_df.filter(regex="_votes$").astype("Int64")
)


# 8th
# change column types to save memory
ratings_df["is_rewatching"] = ratings_df["is_rewatching"].astype("Int8")
ratings_df["anime_id"] = ratings_df["anime_id"].astype("Int32")
ratings_df["score"] = ratings_df["score"].astype("Int8")
ratings_df["num_watched_episodes"] = ratings_df["num_watched_episodes"].astype("Int32")
# drop duplicates keeping the last entry (most recent)
ratings_df.drop_duplicates(subset=['username', 'anime_id'], keep='last', inplace=True)
# there's a username with NaN value, we'll keep it for now as it matches with the profiles_df
# we only have 1 Nan value in "username" in the profiles_df so we can keep it for now
# I could manage it this way: generate a deterministic placeholder, e.g.:
# the maximum existing user_id + 1
# or a specific labeled ID like "unknown_user"
# THEN we can use this placeholder consistently across all datasets to maintain referential integrity.
# THIS won't work though because user_id is missing in both datasets, how can we be sure that the Nan in profiles_df matches the Nan in ratings_df?
# In this case it's easy because there's just one Nan in both datasets, but for future occasions we should do something else?
# 
# profiles_df["user_id"] = profiles_df["user_id"].fillna(new_id)
# ratings_df["user_id"] = ratings_df["user_id"].fillna(new_id)
# TODO

for chunk in pd.read_csv("datasets/ratings.csv", chunksize=1_000_000):
    chunk["is_rewatching"] = chunk["is_rewatching"].astype("Int8")
    chunk["anime_id"] = chunk["anime_id"].astype("Int32")
    chunk["score"] = chunk["score"].astype("Int8")
    chunk["num_watched_episodes"] = chunk["num_watched_episodes"].astype("Int32")
    
    chunk.to_csv(
        "datasets/ratings_cleaned.csv", mode="w" if first_chunk else "a", 
        index=False, header=first_chunk,lineterminator="\n"
    )
    first_chunk = False

ratings_cleaned_df = pd.read_csv("datasets/ratings_cleaned.csv")
ratings_cleaned_df.drop_duplicates(subset=["username", "anime_id"], keep="last", inplace=True)
ratings_cleaned_df.to_csv("datasets/ratings_cleaned.csv", index=False, lineterminator="\n")


# 9th
# change "character_mal_id" and "favorites" from float to int
characters_df[["character_mal_id", "favorites"]] = (
    characters_df[["character_mal_id", "favorites"]].astype("Int64")
)

# we drop the two rows with all columns Nan
characters_df.dropna(how='all', inplace=True)
# we drop the duplicates because they have all same values 
characters_df.drop_duplicates(subset=['character_mal_id', 'url', 'name'], keep='first', inplace=True)


# 10th
# no need to clean


# 11th
person_voice_works_df.drop_duplicates(keep='first', inplace=True)


# 12th
# change column types from object to datetime
profiles_df[["birthday", "joined"]] = profiles_df[["birthday", "joined"]].apply(
    pd.to_datetime, errors="coerce"
)
# setting birthdays outside a reasonable range to NaT
profiles_df.loc[
    profiles_df["birthday"] > profiles_df["joined"],
    "birthday"
] = pd.NaT
# decided to remove the birthdays of people older than 100 at join
mask_too_old = (profiles_df["joined"] - profiles_df["birthday"]).dt.days / 365.25 > 100
profiles_df.loc[mask_too_old, "birthday"] = pd.NaT
#I mean MongoDB, a female born in 1930-07-09, from Thailand and joined in 2017-07-09 makes perfect sense, doesn't it?

# 13th
# no need to clean


### QUESTIONS:
1. Can we just leave the commands to clean the dataset instead of leaving all the exploration commands such as .describe, dtypes and so on
One file with data cleaning, showing the reasoning.
ANOTHER file with Data Visualization.

2. Ask for the maximum age in the 5th dataset
Justify why you have changed the dates (e.g. change 2070 in 1970) or nan. Possiamo fare il cazzo che vogliamo

3. Ask for the only NaN username in the "ratings" dataset: for the hci part?
Think in terms of the future, take the latest id and sum 1, or progressive uknownuserssssssss (e.g. uknown1, unknown2 in caso poi riceviamo altri dati nel futuro)

For TWEB we could just put a random (?) value in order not to lose the line and have non nan values in the database

SE NON USIAMO COLONNE IN ASSOLUTO, per alcuna query o nel sito tweb
Siccome non faremo mai query su questa cosa qui POSSIAMO TOGLIERE le colonne (e.g. kanji name lo possiamo togliere)